<a href="https://colab.research.google.com/github/kingtuler1454/torch/blob/main/5kab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.   Необходимо загрузить исходный набор данных и соответствующие метки классов.
2.   Произвести разделение загруженного набора данных на обучающую, тестовую и валидационую выборки (в соотношении 80:10:10). Проверить, что сформированные выборки сбалансированы.
3.   Написать модель нейронной сети для решения задачи классификации.
4.   Описать пайплайн предобработки данных. **ВАЖНО**: что так как ваш вариант предполагает работу с текстом, то необходимо выполниить векторизацию данных (подробности в туториале).
4.   Написать `train loop` (цикл обучения). Провести эксперименты по обучению с различными значениями параметров `learning rate` (скорость обучения) и `ba
tch size` (размер мини-пакета). Выбрать по 3 значения для `learning rate` и `batch size` (итоговое количество экспериментов будет 9).
5.   Для каждого проведенного эксперимента вывести графики для значения функции потерь (ось `x` - итерация обучения/номер эпохи; ось `y` - значение функции потерь) и выбранной метрики качества (ось `x` - итерация обучения/номер эпохи; ось `y` - значение метрики качества). Графики необходимо выводить как для обучающей, так и для валидационной выборки.
6.   Оценить качество работы модели на тестовой выборке.
7.   Сделайте выводы по полученным результатам проведенных экспериментов. Какую модель из всех полученных стоит использовать?
8.   Сохранить обученную модель.
9.   Выполните повторную инициализацию модели и загрузку весов.  Продемонстрируйте работоспособность модели (пропустите через нее какой-то отзыв/рецензию и выведите результат).


https://www.kaggle.com/code/shivammehta007/spam-not-spam-classifier-with-pytorch

In [ ]:
import pandas as pd
from numpy import loadtxt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
from tqdm import trange

from pymystem3 import Mystem

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import random

In [ ]:
def Lemmatize(src: list):
    ''' Лемматизирует переданный датасет
    '''
    text_nomalized = ' '.join(src).lower() 

    m = Mystem()
    lemmas = m.lemmatize(text_nomalized)
    
    return lemmas

Описать пайплайн предобработки данных. ВАЖНО: что так как ваш вариант предполагает работу с текстом, то необходимо выполниить векторизацию данных (подробности в туториале).


In [ ]:
data = pd.read_csv("dataframe.csv")

data.dropna(inplace=True)
remove_non_alphabets = lambda x: re.sub(r'[^а-яА-Я]',' ',str(x))

tokenize = lambda x: word_tokenize(x, language = "russian")

ps = PorterStemmer()
stem = lambda w: [ ps.stem(x) for x in w ]

print('remove_non_alphabets')
data['text_of_comment'] = data['text_of_comment'].apply(remove_non_alphabets)

print('tokenize')
data['text_of_comment'] = data['text_of_comment'].apply(tokenize) # [ word_tokenize(row) for row in data['email']]

print('stem')
data['text_of_comment'] = data['text_of_comment'].apply(stem)

print('Lemmatize')
c = 0
for words_list in data['text_of_comment']:
    words_list = Lemmatize(words_list)
    print(c)
    c+=1

print('complete')
data['text_of_comment'] = data['text_of_comment'].apply(lambda x: ' '.join(x))

data.dropna(inplace=True)

print(data)


Произвести разделение загруженного набора данных на обучающую, тестовую и валидационую выборки (в соотношении 80:10:10). Проверить, что сформированные выборки сбалансированы.

In [ ]:
frac_seed = random.randint(0,10)
train_df = data.sample(frac=0.9, random_state=frac_seed, ignore_index=True)
valid_df = data.sample(frac=0.1, random_state=frac_seed, ignore_index=True)
for item in valid_df["text_of_comment"]:
    while item in train_df['text_of_comment']: 
        tmp = data.sample()
        item["text_of_comment"] = tmp['text_of_comment']
        item["mark"] = tmp["mark"]
    
print(data)
print(train_df)
print(valid_df)

Написать модель нейронной сети для решения задачи классификации.


In [ ]:
max_words = 10000
stopWords = stopwords.words('russian')

cv = CountVectorizer(max_features=max_words , stop_words=stopWords)

sparse_matrix = cv.fit_transform(train_df["text_of_comment"]).toarray()
valid_sparse_matrix = cv.fit_transform(valid_df["text_of_comment"]).toarray()
print(sparse_matrix.shape)

x_val, x_test, y_val, y_test = train_test_split(
    valid_sparse_matrix, np.array(valid_df["mark"])
)

x_train, x_test, y_train, y_test = train_test_split(
    sparse_matrix, np.array(train_df["mark"],test_size = 0.1, train_size= 0.9)
)

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(10000, 100)
        self.linear2 = nn.Linear(100, 10)
        self.linear3 = nn.Linear(10, 6)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x


#model = LogisticRegression()
#criterion = nn.CrossEntropyLoss()

x_train = torch.Tensor(x_train).float()
y_train = [int(i) for i in y_train]
y_train = torch.Tensor(y_train).long()
x_val = torch.Tensor(x_val).float()
y_val = [int(i) for i in y_val]
y_val = torch.Tensor(y_val).long()

Написать train loop (цикл обучения). Провести эксперименты по обучению с различными значениями параметров learning rate (скорость обучения) и ba tch size (размер мини-пакета). Выбрать по 3 значения для learning rate и batch size (итоговое количество экспериментов будет 9).

Для каждого проведенного эксперимента вывести графики для значения функции потерь (ось x - итерация обучения/номер эпохи; ось y - значение функции потерь) и выбранной метрики качества (ось x - итерация обучения/номер эпохи; ось y - значение метрики качества). 

In [ ]:
def expirement(lr, batch_size):
    model = LogisticRegression()
    criterion = nn.BCELoss()
    
    data = torch.utils.data.DataLoader(x_train, batch_size = batch_size)
    optimizer = torch.optim.Adam(params=model.parameters() , lr = lr)
    epochs = 10
    model.train()
    loss_values = []
    for epoch in range(epochs):
        for elems in data: 
            #optimizer.zero_grad()
            y_pred = model(x_train)
            print(y_pred[batch_size], y_train[batch_size].resize(1).to(dtype=float))
            loss = criterion(y_pred, y_train)
            loss_values.append(loss.item())
            pred = torch.max(y_pred, 1)[1].eq(y_train).sum()
            acc = pred * 100.0 / len(x_train)
            print('Epoch: {}, Loss: {}, Accuracy: {}%'.format(epoch+1, loss.item(), acc.numpy()))
            loss.backward()
            optimizer.step()
    plt.plot(loss_values)
    plt.title('Таблица эпох и потерь при lr='+str(lr) +" batch_size="+str(batch_size))
    plt.xlabel('Эпоха')
    plt.ylabel('Потери')
    plt.legend(['Потери'])
    plt.show()
    
if True:
    
    lr_list=[0.0001,0.0002,0.0003]
    batch_size=[1000,500,100]

    for LR in lr_list:
            for BZ in batch_size:
                expirement(LR,BZ)

Оценить качество работы модели на тестовой выборке.

In [ ]:
x_test = torch.Tensor(x_test).float()
y_test = [int(i) for i in y_test]
y_test = torch.Tensor(y_test).long()
model.eval()
with torch.no_grad():
    for i in range(20):
        y_pred = model(x_test)
        loss = criterion(y_pred, y_test)
        pred = torch.max(y_pred, 1)[1].eq(y_test).sum()
    print("Accuracy : {}%".format(100 * pred / len(x_test)))

Сделайте выводы по полученным результатам проведенных экспериментов. Какую модель из всех полученных стоит использовать?

In [ ]:
LR = 0.01
BZ = 100

Сохранить обученную модель.

In [ ]:
torch.save(model.state_dict(), os.path.join("/content/", "weight.pt"))

Выполните повторную инициализацию модели и загрузку весов. Продемонстрируйте работоспособность модели (пропустите через нее какой-то отзыв/рецензию и выведите результат).

In [ ]:
model.load_state_dict(torch.load(os.path.join("/content/", "weight.pt")))
data = torch.utils.data.DataLoader(x_test, batch_size=BZ, shuffle=True)
model.eval()
text_list = []
with torch.no_grad():
    for i in range(20):
        y_pred = model(x_test)
        loss = criterion(y_pred, y_test)
        pred = torch.max(y_pred, 1)[1].eq(y_test).sum()
    print("Accuracy : {}%".format(100 * pred / len(x_test)))
